In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/covid19-local-us-ca-forecasting-week-1/ca_train.csv')
print(train.columns)
test = pd.read_csv('/kaggle/input/covid19-local-us-ca-forecasting-week-1/ca_test.csv')
submission = pd.read_csv('/kaggle/input/covid19-local-us-ca-forecasting-week-1/ca_submission.csv')
sub = submission['ForecastId']
print(sub)

In [ ]:
train.rename(columns={'Province/State':'Province','Country/Region':'Region'},inplace=True)
print(train.columns)

In [ ]:
train.drop(columns='Province',inplace=True)
print(train.columns)

In [ ]:
print(train.head())

Divide the data into parts to predict the output

In [ ]:
train['Date'] = train['Date'].apply(lambda x:x.replace("-",""))
train['Date'] = train['Date'].astype('int')
test['Date'] = test['Date'].apply(lambda x:x.replace("-",""))
test['Date'] = test['Date'].astype('int')

In [ ]:
X = train[['Lat','Long','Date']]
y1= train['ConfirmedCases']
y2= train['Fatalities']
X_test = test[['Lat','Long','Date']]

from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier()
model.fit(X,y1)
y_pred=model.predict(X_test)
y_pred =pd.DataFrame(y_pred)
y_pred.columns = ["ConfirmedCases_prediction"]
print(y_pred.head())

In [ ]:
model.fit(X,y2)
y_pred1=model.predict(X_test)
y_pred1 =pd.DataFrame(y_pred1)
y_pred1.columns = ["Death_Prediction"]
print(y_pred1.head())

In [ ]:
submit = pd.concat([y_pred,y_pred1,sub],axis=1)
print(submit.head())

In [ ]:
submit.columns = ['ConfirmedCases', 'Fatalities', 'ForecastId']
submit = submit[['ForecastId','ConfirmedCases', 'Fatalities']]

submit["ConfirmedCases"] = submit["ConfirmedCases"].astype(int)
submit["Fatalities"] = submit["Fatalities"].astype(int)

In [ ]:
sub = submit
sub.to_csv('submission.csv', index=False)